# 2020-2022 date predict

In [1]:
# text_20_22 = pd.read_csv("data/cleaned_projects_20_22.csv")
# print(text_20_22)

In [2]:
# print(text_20_22.columns)
# print(text_20_22.shape)

In [3]:
# from nltk.tokenize import sent_tokenize

# # 1. 过滤非空 description
# df = text_20_22[text_20_22['description'].notna()]

# # 2. 清洗多余空格
# df['description'] = df['description'].str.replace(r'\s+', ' ', regex=True).str.strip()

# # 3. 拆分为句子
# df['sentences'] = df['description'].apply(sent_tokenize)

# # 4. 扁平化为句子列表
# all_sentences_20_22 = df['sentences'].explode().tolist()

# # 5. 编码
# sentence_vecs_20_21 = sbert_model.encode(all_sentences_20_22, show_progress_bar=True, batch_size=32, normalize_embeddings=True)


In [4]:
# print(sentence_vecs_20_21)

In [5]:
# # DataFrame + merge back to main list
# vec_df = pd.DataFrame(sentence_vecs_20_21)
# vec_df.columns = [f'sbert_{i}' for i in range(vec_df.shape[1])]
# df_vectors = pd.concat([df.reset_index(drop=True), vec_df], axis=1)
# print(df_vectors.head())

In [6]:
# df_vectors.to_parquet('sbert_encoded_data_22.parquet')

开始测试2020-2022

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import Polygon

In [2]:
import joblib

# 加载模型
kmeans = joblib.load("kmean_model_original.pkl")

In [3]:
lsoa_label_22 = gpd.read_file("data/gentri_data/all_stru_data_20_22.shp")
lsoa_label_22.crs

<Projected CRS: EPSG:27700>
Name: OSGB36 / British National Grid
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: United Kingdom (UK) - offshore to boundary of UKCS within 49°45'N to 61°N and 9°W to 2°E; onshore Great Britain (England, Wales and Scotland). Isle of Man onshore.
- bounds: (-9.01, 49.75, 2.01, 61.01)
Coordinate Operation:
- name: British National Grid
- method: Transverse Mercator
Datum: Ordnance Survey of Great Britain 1936
- Ellipsoid: Airy 1830
- Prime Meridian: Greenwich

In [6]:
sbert_data_22 = pd.read_parquet("sbert_encoded_data_22.parquet")
print(sbert_data_22.columns)

Index(['site_name', 'decision_date', 'valid_date', 'polygon', 'wgs84_polygon',
       'description', 'borough', 'street_name', 'polygon.geometries',
       'polygon.type',
       ...
       'sbert_374', 'sbert_375', 'sbert_376', 'sbert_377', 'sbert_378',
       'sbert_379', 'sbert_380', 'sbert_381', 'sbert_382', 'sbert_383'],
      dtype='object', length=401)


In [9]:
print(kmeans.cluster_centers_.dtype)

float64


In [12]:
target_dtype = kmeans.cluster_centers_.dtype  # 自动匹配类型

sbert_columns = [f"sbert_{i}" for i in range(384)]
n_samples = len(sbert_data_22)

# 创建 float64（或 float32）矩阵
X_np = np.zeros((n_samples, 384), dtype=target_dtype)

for i, col in enumerate(sbert_columns):
    X_np[:, i] = sbert_data_22[col].values.astype(target_dtype)

# ✅ 最终 predict 不再报错
sbert_data_22["cluster"] = kmeans.predict(X_np)

e:\dissertation\project\housing_feature_gentrification\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


In [14]:
print(sbert_data_22.columns)

Index(['site_name', 'decision_date', 'valid_date', 'polygon', 'wgs84_polygon',
       'description', 'borough', 'street_name', 'polygon.geometries',
       'polygon.type',
       ...
       'sbert_375', 'sbert_376', 'sbert_377', 'sbert_378', 'sbert_379',
       'sbert_380', 'sbert_381', 'sbert_382', 'sbert_383', 'cluster'],
      dtype='object', length=402)


In [ ]:
# 统计每个 LSOA 中各个 cluster 的数量
cluster_counts = (
    sbert_data_22
    .groupby(["lsoa_code", "cluster"])
    .size()
    .unstack(fill_value=0)
)

# 转为占比（按行标准化）
cluster_props = cluster_counts.div(cluster_counts.sum(axis=1), axis=0)

KeyError: 'lsoa_code'